# Introduction

This is an implementation of [Deep Q-Network](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf), a deep learning variant of the Q-learning algorithm for machine learning, developed by [Google DeepMind](https://deepmind.com/). A very good explanation of Reinforcement Learning, Q-Networks and particular Deep Q-Network can be found [here](https://towardsdatascience.com/introduction-to-various-reinforcement-learning-algorithms-i-q-learning-sarsa-dqn-ddpg-72a5e0cb6287).  
The implemention is inspired and partially based on [this one](https://github.com/jaidmin/pytorch-q-learning). Besides I used some ideas from [this repository](https://github.com/Shmuma/ptan/tree/master/samples/dqn_speedup) and Atari wrappers from [OpenAI Baselines repositories](https://github.com/openai/baselines/).

## Imports

In [1]:
%matplotlib inline

import time

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from copy import deepcopy
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as T

## Initialization

In [13]:
use_gpu = torch.cuda.is_available()
print('Use GPU: {}'.format(use_gpu))

Use GPU: False


In [14]:
use_mps = torch.backends.mps.is_available()
print('Use MPS: {}'.format(use_mps))

Use MPS: True


## Agent

In [ ]:
from train_dqn import *

In [18]:
agent = PongAgent()

/Users/jesse/miniconda3/envs/pong_dqn/lib/python3.9/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Load the model and play

In [7]:
agent.load_model('./model/final_model.pth')
agent.play(3)

In [8]:
agent.close_env()

In [20]:
agent.play(3)

In [21]:
agent.close_env()